# Connector

In [4]:
import requests
import pandas as pd

import os
from dotenv import load_dotenv

load_dotenv()

Alpaca_endpoint = os.getenv("ALPACA_ENDPOINT")
Alpaca_key = os.getenv("ALPACA_KEY")
Alpaca_secret = os.getenv("ALPACA_SECRET")


# connect to alphca api

def connect_to_alphca_api(api_key: str) -> requests.Session:
    session = requests.Session()
    session.headers.update({
        "Authorization": f"Bearer {api_key}"
    })
    return session

print(connect_to_alphca_api(Alpaca_key))

# Order Book backtest

In [ ]:
import json
from datetime import datetime

# Load the entire JSON file
with open('order_book.json', 'r') as f:
    orderbook_data = json.load(f)s


# Access a single
first_snapshot = orderbook_data[0]
print(f"Asset: {first_snapshot['asset']}")
print(f"Time: {first_snapshot['time']}")
print(f"Best Bid: {first_snapshot['data']['bids'][0]['price']}")
print(f"Best Ask: {first_snapshot['data']['asks'][0]['price']}")

for snapshot in orderbook_data[0:10]:
    print(snapshot['time'])


Asset: BTC/USD
Time: 2025-11-23T22:33:25.01579709Z
Best Bid: 85221.674876847
Best Ask: 87826.6
2025-11-23T22:33:25.01579709Z
2025-11-23T22:33:25.308409767Z
2025-11-23T22:33:25.308464977Z
2025-11-23T22:33:25.308475747Z
2025-11-23T22:33:25.308563707Z
2025-11-23T22:33:25.308574887Z
2025-11-23T22:33:25.308617297Z
2025-11-23T22:33:25.308689717Z
2025-11-23T22:33:25.308789997Z
2025-11-23T22:33:25.308814407Z


# Position

In [6]:
from utils.account_info import AccountInfo
account = AccountInfo()

print(account.get_account_balance())
print(account.get_buying_power())
print(account.get_equity())
print(account.get_portfolio_value())

# position objects
for position in account.get_positions():
    print(position.symbol)

# access with object
print(account.get_position("NVDA").symbol)
print(account.has_position("BTC/USD"))

print(account.get_all_position_symbols())
print(account.get_total_market_value())
print(account.get_total_unrealized_pl())

97242.15
194663.18
100006.13
100006.13
BTCUSD
NVDA
NVDA
False
['BTCUSD', 'NVDA']
2763.9810500000003
-2.61757


# order manager

In [ ]:
from order_manager import OrderManager

order_manager = OrderManager()

order_manager.buy(symbol="BTC/USD", limit_price=100000, quantity=0.01)
# returs response -> can parse order status

buy 0.01 BTC/USD @ 100000


{'id': '57ecbd1f-1215-4678-86fd-cacfe5c01289',
 'client_order_id': 'cdd973c4-0e8f-4e46-87b9-b640008df6e2',
 'created_at': '2025-11-24T00:32:21.052627114Z',
 'updated_at': '2025-11-24T00:32:21.053838023Z',
 'submitted_at': '2025-11-24T00:32:21.052627114Z',
 'filled_at': None,
 'expired_at': None,
 'canceled_at': None,
 'failed_at': None,
 'replaced_at': None,
 'replaced_by': None,
 'replaces': None,
 'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
 'symbol': 'BTC/USD',
 'asset_class': 'crypto',
 'notional': None,
 'qty': '0.01',
 'filled_qty': '0',
 'filled_avg_price': None,
 'order_class': '',
 'order_type': 'limit',
 'type': 'limit',
 'side': 'buy',
 'position_intent': 'buy_to_open',
 'time_in_force': 'ioc',
 'limit_price': '100000',
 'stop_price': None,
 'status': 'pending_new',
 'extended_hours': False,
 'legs': None,
 'trail_percent': None,
 'trail_price': None,
 'hwm': None,
 'subtag': None,
 'source': None,
 'expires_at': '2025-11-21T21:00:00Z'}

# execution engine


In [ ]:
import asyncio
from utils.events import Event, EventType, Signal
from utils.execution_engine import ExecutionEngine
from order_manager import OrderManager

# Create a queue
signal_q = asyncio.Queue()

# Create order manager
order_manager = OrderManager()

# Create execution engine
exec_engine = ExecutionEngine(
    in_q=signal_q,
    order_manager=order_manager
)

# Create a test signal
test_signal = Signal(
    action="buy",
    symbol="BTC/USD",
    limit_price=100000.0,
    quantity=0.001
)

# Create an event with the signal
signal_event = Event(
    type=EventType.SIGNAL,
    payload=test_signal
)

# Put the event in the queue
await signal_q.put(signal_event)

# Run the execution engine
task = asyncio.create_task(exec_engine.run())

# Wait a bit for it to process
await asyncio.sleep(0.1)

# Cancel the task
task.cancel()
try:
    await task
except asyncio.CancelledError:
    pass

buy 0.01 BTC/USD @ 100000.0


# OrderBook

In [2]:
from utils.order_book import OrderBook
from sortedcontainers import SortedDict

orderbook = OrderBook(symbol="BTC/USD", max_levels=10, trim_frequency=100)

orderbook.asks = SortedDict({
    103: 27,
    102: 50,
    101: 80
})

orderbook.bids = SortedDict({
    -99: 40,
    -98: 200,
    -97: 67
})

orderbook.print_orderbook(full = True)



Ask: (103, 27)
Ask: (102, 50)
Ask: (101, 80)
Best Ask: 101, 80
--------------------------------
Spread: $2 , Mid Price: $100.0
--------------------------------
Best Bid: 99, 40
Bid: (99, 40)
Bid: (98, 200)
Bid: (97, 67)




# Strategy runner

In [1]:
import asyncio
from utils.events import Event, EventType
from Strats.strategy_runner import StrategyRunner
from order_manager import OrderManager
from Strats.ob_imbalance import OrderBookImbalanceStrategy
from utils.order_book import OrderBook
from sortedcontainers import SortedDict
from utils.execution_engine import ExecutionEngine

# Create queues
orderbook_q = asyncio.Queue()
signal_q = asyncio.Queue()

order_manager = OrderManager()

strategy = OrderBookImbalanceStrategy(order_manager=order_manager)

runner = StrategyRunner(
    in_q=orderbook_q,
    out_q=signal_q,
    strategy=strategy
)

# exec_engine = ExecutionEngine(
#     in_q=signal_q,
#     order_manager=order_manager
# )

test_orderbook = OrderBook(symbol="BTC/USD", max_levels=10, trim_frequency=100)

test_orderbook.asks = SortedDict({
    103: 27,
    102: 50,
    101: 80
})
test_orderbook.bids = SortedDict({
    -99: 40,
    -98: 200,
    -97: 67
})

# Create an event with the signal
signal_event = Event(
    type=EventType.ORDERBOOK_UPDATE,
    payload=test_orderbook
)


# Put the event in the queue
await orderbook_q.put(signal_event)
await signal_q.put(signal_event)

# Run the execution engine
# task = asyncio.create_task(exec_engine.run())
task = asyncio.create_task(runner.run())

# Wait a bit for it to process
await asyncio.sleep(0.1)

# Cancel the task
task.cancel()
try:
    await task
except asyncio.CancelledError:
    pass

# The order is not executed because the execution engine is not running...


on_data
ob_imbalance: buy signal
strategy runner: signal


# Trade rule test